In [ ]:
from comet import download_model, load_from_checkpoint
import csv
import numpy as np
import pandas as pd

In [ ]:
# model_path = download_model("Unbabel/wmt22-cometkiwi-da", saving_directory="models/cometkiwi_wmt22") # for reference-free evaluation
model_path = "models/cometkiwi_wmt22/models--Unbabel--wmt22-cometkiwi-da/snapshots/265631cbbaf0740fc94aee7901930817dcc853c7/checkpoints/model.ckpt"

Fetching 5 files: 100%|██████████| 5/5 [00:00<00:00, 18.63it/s]


In [38]:
print(model_path)

models/cometkiwi_wmt22\models--Unbabel--wmt22-cometkiwi-da\snapshots\265631cbbaf0740fc94aee7901930817dcc853c7\checkpoints\model.ckpt


In [39]:
model = load_from_checkpoint(model_path)

Lightning automatically upgraded your loaded checkpoint from v1.8.2 to v2.5.0.post0. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint c:\Users\antoi\OneDrive\Documents\UofT Fall 2024\ESC499 - Thesis\github\translation_thesis\translation_analysis\analysis1\models\cometkiwi_wmt22\models--Unbabel--wmt22-cometkiwi-da\snapshots\265631cbbaf0740fc94aee7901930817dcc853c7\checkpoints\model.ckpt`
c:\Users\antoi\OneDrive\Documents\UofT Fall 2024\ESC499 - Thesis\github\translation_thesis\venv\lib\site-packages\huggingface_hub\file_download.py:142: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\antoi\.cache\huggingface\hub\models--microsoft--infoxlm-large. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING

In [ ]:
def read_csv_to_numpy(csv_file_path):
    try:
        # Read the CSV with utf-8-sig encoding to handle the BOM
        df = pd.read_csv(csv_file_path, encoding='utf-8-sig')
        
        # Convert the DataFrame to a NumPy array
        numpy_array = df.to_numpy()
        return numpy_array
    except FileNotFoundError:
        print(f"Error: The file '{csv_file_path}' does not exist.")
        return None
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

# Example usage
csv_file_path = '../../data_collection_app/app_latest_database/translation.csv'
numpy_array = read_csv_to_numpy(csv_file_path)

# Print the NumPy array
if numpy_array is not None:
    print(numpy_array)


[[1 'eng_to_spa' 'You run' 'Tu corres' 'Tú corres.' 2
  '2024-11-23 19:01:39']
 [2 'eng_to_spa' 'Yes' 'Sí' 'Sí' 3 '2024-11-23 19:01:50']
 [3 'spa_to_eng' 'tu esposa te regañaba por no recoger a sus niños'
  'Your wife scolded you for not picking up your children'
  'Your wife would scold you for not picking up her kids.' 1
  '2024-11-23 19:38:39']
 [4 'spa_to_eng' 'Obtén un batidor. Bate las fresas.'
  'Get a mixer. Blend the strawberries.'
  'Get a whisk. Whisk the strawberries.' 1 '2024-11-23 19:39:28']
 [7 'eng_to_spa' 'Barcelona beat Real Madrid'
  'El Barcelona venció al Real Madrid'
  'El Barcelona venció al Real Madrid.' 3 '2024-11-23 19:42:17']
 [8 'spa_to_eng' 'un callejón' 'an alley' 'a alleyway' 1
  '2024-11-24 01:25:15']
 [9 'eng_to_spa' 'you run' 'tu corres' 'tú corres' 2
  '2024-11-26 21:29:20']
 [10 'eng_to_spa' 'Hello how are you?' '¿Hola, cómo estás?'
  'Hola, ¿cómo estás?' 3 '2024-11-26 21:29:53']
 [11 'eng_to_spa' 'my head hurts' 'Me duele la cabeza'
  'Me duele la c

In [41]:
print(numpy_array[0][2])
print(numpy_array[0][3])
print(numpy_array[0][4])

You run
Tu corres
Tú corres.


In [42]:
data = [
    {
        "src": numpy_array[0][2],
        "mt": numpy_array[0][3],
    },
    {
        "src": numpy_array[0][2],
        "mt": numpy_array[0][4],
    }
]
# Call predict method:
model_output = model.predict(data, batch_size=8, gpus=0) # quite rapid

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
Predicting DataLoader 0: 100%|██████████| 1/1 [00:05<00:00,  5.85s/it]


In [43]:
print(model_output)

Prediction([('scores', [0.6030316352844238, 0.6196182370185852]), ('system_score', 0.6113249361515045)])
